# **Data Preparation**

So, we have our .csv files of the songs, one time with 1000, the other time with 5000 songs and what we maybe come up with later to have fun.
Our learning platform also provides us with further information about the columns:

**acousticness** - A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. - *Float*  
**danceability** - Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.	- *Float*  
**duration_ms** - The duration of the track in milliseconds. - *Integer*  
**energy** - Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. - *Float*  
**instrumentalness** - Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. - *Float*  
**key** - The key the track is in. Integers map to pitches using standard Pitch Class notation . E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. - *Integer*  
**liveness** Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. - *Float*  
**loudness** - The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db. - *Float*  
**mode** - Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. - *Integer*  
**speechiness** - Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. - *Float*  
**tempo** - The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. - *Float*  
**time_signature** - An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). - *Integer*  
**valence** - A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).	*Float*  

We now have to choose which columns are worth to keep based on these informations and our own thoughts about it and store them in a new dataframe we even might want to turn into a .csv.

## **1. Importing the needed modules and .csv files**

In [ ]:
# We want pandas and our scaler:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
# We keep .csv files in same path as our project so we easy navigate to them:
df_1000 = pd.read_csv(r'df_audio_features_1000.csv')
df_5000 = pd.read_csv(r'df_audio_features_5000.csv',skipinitialspace=True)


## **2. Inspecting the DataFrame to Clean it** 

In [ ]:
#Basic pandas stuff:
# df_1000.info() # Put/Remove comment to read/see
# df_1000.sample(5) # Put/Remove comment to read/see
# df_5000.sample(5) # Put/Remove comment to read/see
# df_5000['duration_ms'].mean() #reasoning for smaller clusters later on...


### Conclusion here:
* we want to combine artist and name into one column (might be covers on some datasets) 
* drop the other 2 columns with Dtype = object ('id','html'), we need only numbers...
* drop 'key', 'mode', 'type', 'duration_ms', 'time_signature', since they seem meaningless for us.

## **3. Preparing the data:**
### **3.1 1000 songs df:**

In [ ]:
# New column 'title' to get rid of 2 objects:
df_1000['title'] = df_1000['name'] + ' - ' + df_1000['artist']
# Drop a bunch of stuff:
# df_1000.drop(['artist','name','id','html','type','duration_ms']) # cant find the freaking typo - can aswell create a sorted:
clustersongs_1000 = df_1000[['title','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','time_signature','valence','tempo']]
# clustersongs_1000.sample(1) # See how it worked
clustersongs_1000.set_index('title', inplace=True)
# convert into (unmodified) .csv to work with in exploration:
clustersongs_1000.to_csv('1000songs.csv')


### **3.2 5000 songs df**:

In [ ]:
df_5000.columns = df_5000.columns.str.strip()
df_5000['title'] = df_5000['name'] + ' - ' + df_5000['artist']
clustersongs_5000 = df_5000[['title','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','time_signature','valence','tempo']]
clustersongs_5000.set_index('title', inplace=True)
clustersongs_5000.to_csv('5000songs.csv')

## **4. Different Scaler:**

### **4.1 MinMax:**

In [ ]:
# Taken from learning platform:
# initialize the transformer (optionally, set parameters)
my_min_max = MinMaxScaler(feature_range=(0,1)) 
# fit the transformer to the data
my_min_max.fit(clustersongs_1000)
# use the transformer to transform the data
minmax_1000_df = my_min_max.transform(clustersongs_1000) 
# reconvert the transformed data back to a DataFrame
minmax_1000 = pd.DataFrame(minmax_1000_df,
              index=clustersongs_1000.index,
              columns=clustersongs_1000.columns)
# make .csv for next notebook
minmax_1000.to_csv('1000min_max.csv')
minmax_1000.sample(5)


### **4.2 Standart:**

In [ ]:
standart = StandardScaler(with_mean=True, with_std=True).set_output(transform='pandas')
scaled_1000 = standart.fit_transform(clustersongs_1000)
scaled_1000.to_csv('1000standart.csv')
scaled_1000.sample(5)

### **4.3 Robust:**

In [ ]:
robust = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0), copy=True, unit_variance=False).set_output(transform='pandas')
robust_1000 = robust.fit_transform(clustersongs_1000)
robust_1000.to_csv('1000robust.csv')
robust_1000.sample(5)

### **4.4 Quantile:**

In [ ]:
quantile = QuantileTransformer(n_quantiles=10, output_distribution='uniform', ignore_implicit_zeros=False, subsample=10000, random_state=None, copy=True).set_output(transform='pandas')
quantile_1000 = quantile.fit_transform(clustersongs_1000)
quantile_1000.to_csv('1000quantile.csv')
quantile_1000.sample(5)

### **4.5 Power:**

In [ ]:
power = PowerTransformer(method='yeo-johnson', standardize=True, copy=True).set_output(transform='pandas')
power_1000 = power.fit_transform(clustersongs_1000)
power_1000.to_csv('1000power.csv')
power_1000.sample(5)

## **5. Different Scaler for 5000 songs:**

In [ ]:
# Taken from above with 1000 songs:
# MinMax:
my_min_max.fit(clustersongs_5000)
minmax_5000_df = my_min_max.transform(clustersongs_5000) 
minmax_5000 = pd.DataFrame(minmax_5000_df,
              index=clustersongs_5000.index,
              columns=clustersongs_5000.columns)
minmax_5000.to_csv('5000min_max.csv')
# Standart:
scaled_5000 = standart.fit_transform(clustersongs_5000)
scaled_5000.to_csv('5000standart.csv')
# Robust:
robust_5000 = robust.fit_transform(clustersongs_5000)
robust_5000.to_csv('5000robust.csv')
# Quantile:
quantile_5000 = quantile.fit_transform(clustersongs_5000)
quantile_5000.to_csv('5000quantile.csv')
# Power:
power_5000 = power.fit_transform(clustersongs_5000)
power_5000.to_csv('5000power.csv')
